## Testowanie działania klasyfikatorów na trójce polskich pisarzy


Importujemy potrzebne biblioteki

In [33]:
import os, fnmatch
import pandas as pd

Dodajemy implementację potrzebnego klasyfikatora

In [34]:
class TxtDocMatrixClassifier:
    __unnecessary_chars = ["!",",",".","    ",":",";","?","(",")","—","*","»","…","«","—","-","„","”",'"',"°","'"] 
    __stops = ['a', 'aby', 'ach', 'acz', 'aczkolwiek', 'aj', 'albo', 'ale', 'alez', 'ależ', 'ani', 'az', 'aż', 'bardziej', 'bardzo', 'beda', 'bedzie', 'bez', 'deda', 'będą', 'bede', 'będę', 'będzie', 'bo', 'bowiem', 'by', 'byc', 'być', 'byl', 'byla', 'byli', 'bylo', 'byly', 'był', 'była', 'było', 'były', 'bynajmniej', 'cala', 'cali', 'caly', 'cała', 'cały', 'ci', 'cie', 'ciebie', 'cię', 'co', 'cokolwiek', 'cos', 'coś', 'czasami', 'czasem', 'czemu', 'czy', 'czyli', 'daleko', 'dla', 'dlaczego', 'dlatego', 'do', 'dobrze', 'dokad', 'dokąd', 'dosc', 'dość', 'duzo', 'dużo', 'dwa', 'dwaj', 'dwie', 'dwoje', 'dzis', 'dzisiaj', 'dziś', 'gdy', 'gdyby', 'gdyz', 'gdyż', 'gdzie', 'gdziekolwiek', 'gdzies', 'gdzieś', 'go', 'i', 'ich', 'ile', 'im', 'inna', 'inne', 'inny', 'innych', 'iz', 'iż', 'ja', 'jak', 'jakas', 'jakaś', 'jakby', 'jaki', 'jakichs', 'jakichś', 'jakie', 'jakis', 'jakiś', 'jakiz', 'jakiż', 'jakkolwiek', 'jako', 'jakos', 'jakoś', 'ją', 'je', 'jeden', 'jedna', 'jednak', 'jednakze', 'jednakże', 'jedno', 'jego', 'jej', 'jemu', 'jesli', 'jest', 'jestem', 'jeszcze', 'jeśli', 'jezeli', 'jeżeli', 'juz', 'już', 'kazdy', 'każdy', 'kiedy', 'kilka', 'kims', 'kimś', 'kto', 'ktokolwiek', 'ktora', 'ktore', 'ktorego', 'ktorej', 'ktory', 'ktorych', 'ktorym', 'ktorzy', 'ktos', 'ktoś', 'która', 'które', 'którego', 'której', 'który', 'których', 'którym', 'którzy', 'ku', 'lat', 'lecz', 'lub', 'ma', 'mają', 'mało', 'mam', 'mi', 'miedzy', 'między', 'mimo', 'mna', 'mną', 'mnie', 'moga', 'mogą', 'moi', 'moim', 'moj', 'moja', 'moje', 'moze', 'mozliwe', 'mozna', 'może', 'możliwe', 'można', 'mój', 'mu', 'musi', 'my', 'na', 'nad', 'nam', 'nami', 'nas', 'nasi', 'nasz', 'nasza', 'nasze', 'naszego', 'naszych', 'natomiast', 'natychmiast', 'nawet', 'nia', 'nią', 'nic', 'nich', 'nie', 'niech', 'niego', 'niej', 'niemu', 'nigdy', 'nim', 'nimi', 'niz', 'niż', 'no', 'o', 'obok', 'od', 'około', 'on', 'ona', 'one', 'oni', 'ono', 'oraz', 'oto', 'owszem', 'pan', 'pana', 'pani', 'po', 'pod', 'podczas', 'pomimo', 'ponad', 'poniewaz', 'ponieważ', 'powinien', 'powinna', 'powinni', 'powinno', 'poza', 'prawie', 'przeciez', 'przecież', 'przed', 'przede', 'przedtem', 'przez', 'przy', 'roku', 'rowniez', 'również', 'sam', 'sama', 'są', 'sie', 'się', 'skad', 'skąd', 'soba', 'sobą', 'sobie', 'sposob', 'sposób', 'swoje', 'ta', 'tak', 'taka', 'taki', 'takie', 'takze', 'także', 'tam', 'te', 'tego', 'tej', 'ten', 'teraz', 'też', 'to', 'toba', 'tobą', 'tobie', 'totez', 'toteż', 'totobą', 'trzeba', 'tu', 'tutaj', 'twoi', 'twoim', 'twoj', 'twoja', 'twoje', 'twój', 'twym', 'ty', 'tych', 'tylko', 'tym', 'u', 'w', 'wam', 'wami', 'was', 'wasz', 'wasza', 'wasze', 'we', 'według', 'wiele', 'wielu', 'więc', 'więcej', 'wlasnie', 'właśnie', 'wszyscy', 'wszystkich', 'wszystkie', 'wszystkim', 'wszystko', 'wtedy', 'wy', 'z', 'za', 'zaden', 'zadna', 'zadne', 'zadnych', 'zapewne', 'zawsze', 'ze', 'zeby', 'zeznowu', 'zł', 'znow', 'znowu', 'znów', 'zostal', 'został', 'żaden', 'żadna', 'żadne', 'żadnych', 'że', 'żeby']

    def __findBooksPaths(self) -> list:
        result = []
        for root, dirs, files in os.walk(self.__booksRelPath):
            for name in files:
                if fnmatch.fnmatch(name, '*_*.txt'):
                    result.append(os.path.join(root, name))
        return result 

    def __retrieveTokensFromBook(self, path) -> None:
        usidx = path.index('_', len(self.__booksRelPath)+1)
        author = path[len(self.__booksRelPath)+1:usidx]

        self.__authorsWB[author] = self.__authorsWB.get(author, dict())
        tokens_stop = []
        tokens = []

        with open(path, 'r', encoding='utf-8') as book:
            for line in book:        
                for char in self.__unnecessary_chars:
                    line = line.replace(char, '')
                line = line.rstrip().lower().split()

                if line:
                    tokens_stop.extend(line)

        for ts in tokens_stop:
            if not ts in self.__stops:
                tokens.append(ts)

        for word in sorted(tokens):
            if word.isalpha():
                self.__authorsWB[author][word] = self.__authorsWB[author].get(word, 0) + 1

    def __truncateAuthorsWB(self) -> None:
        for author in self.__authorsWB:
            tempWB = dict(sorted(self.__authorsWB[author].items(), key=lambda x: x[1], reverse = True))
            self.__authorsWB[author] = dict()

            count = min(len(tempWB.keys()), self.__lim)
            for token in tempWB:
                self.__authorsWB[author][token] = tempWB[token]

                if count < 0: break
                count -= 1
                
    def __buildAuthorsWB(self) -> None:
        paths = self.__findBooksPaths()
        for p in paths:
            self.__retrieveTokensFromBook(p)
        self.__truncateAuthorsWB()

    def __buildFragDict(self, frag) -> None:
        frag = frag.replace('\n', ' ')
        for char in self.__unnecessary_chars:
            frag = frag.replace(char, '')
        frag = frag.strip().lower().split()

        self.__fragTokens = dict()
        for token in frag:
            if token not in self.__stops:
                self.__fragTokens[token] = self.__fragTokens.get(token, 0) + 1
        self.__fragTokens = dict(sorted(self.__fragTokens.items(), key=lambda x: x[1], reverse = True))

    def __buildTDMfromFrag(self) -> None:
        self.__fragTDM = pd.DataFrame(columns=self.authors)
        for ft in self.__fragTokens:
            licznik = [0] * len(self.authors)
            for i, author in enumerate(self.authors):
                for token in self.__authorsWB[author]:
                    if ft == token:
                        licznik[i] += self.__authorsWB[author][token]
            element = pd.DataFrame([licznik], columns=self.authors, index=[ft])
            self.__fragTDM = self.__fragTDM.append(element)
        
    def classifyFullProb(self, frag) -> dict:
        self.__buildFragDict(frag)
        self.__buildTDMfromFrag()
        
        result = dict()
        for i, el in enumerate(self.__fragTDM.sum()):
            result[self.authors[i]] = el
            
        return result

    def classify(self, frag) -> str:
        full = self.classifyFullProb(frag)
        maxa = ""
        maxi = 0
        for author in full:
            if full[author] > maxi:
                maxi = full[author]
                maxa = author
        return maxa
    
    def getAuthors(self) -> list:
        return self.authors
    
#     def getAuthWB(self):
#         return self.__authorsWB
    
#     def getFragDict(self):
#         return self.__fragTokens

#     def getTDM(self):
#         return self.__fragTDM

    def __init__(self, bookRelPath, lim) -> None:
        self.__booksRelPath = bookRelPath
        self.__authorsWB = dict()
        self.__lim = lim

        self.__buildAuthorsWB()
        self.authors = list(self.__authorsWB.keys())


## Następnie przechodzimy do klasyfikacji i testowania klasyfikatora

Tworzymy klasifikator na podstawie, którego będziemy sprawdzać fragmenty tekstów.

In [35]:
K = TxtDocMatrixClassifier('..\\test_books\\mickiewicz_sienkiewicz_slowacki_books', 100)

Zaczniemy od wyszukania słów w miarę oczywistych i charakterystycznych dla danego autora.

In [36]:
fragment = 'Tadeusz'
print(K.classifyFullProb(fragment))

{'adam-mickiewicz': 107, 'henryk-sienkiewicz': 0, 'juliusz-slowacki': 0}


In [37]:
fragment = 'Balladyna'
print(K.classifyFullProb(fragment))

{'adam-mickiewicz': 0, 'henryk-sienkiewicz': 0, 'juliusz-slowacki': 273}


In [38]:
fragment = 'Zbyszko'
print(K.classifyFullProb(fragment))

{'adam-mickiewicz': 0, 'henryk-sienkiewicz': 957, 'juliusz-slowacki': 0}


Wyniki są póki co bardzo dobre, program bez problemu poradził sobie z klasyfikacją podanych fraz do odpowidnich autorów.

Teraz sprawdźmy nasz klasyfikator dla całych zdań wyjętych z książek.

In [40]:
#Fragment książki W Pustyni i w Puszczy Sienkiewicza
fragment = 'Staś, który kończył rok czternasty i który swą ośmioletnią towarzyszkę kochał bardzo'
print(K.classifyFullProb(fragment))

{'adam-mickiewicz': 0, 'henryk-sienkiewicz': 631, 'juliusz-slowacki': 0}


In [41]:
#Fragment książki Krzyżacy Sienkiewicza
fragment = 'Jurand, znalazłszy się na podwórzu zamkowym, nie wiedział zrazu, dokąd iść, gdyż knecht, który go przeprowadził przez bramę, opuścił go i udał się ku stajniom. Przy blankach stali wprawdzie żołdacy, to pojedynczo, to po kilku razem, ale twarze ich były tak zuchwałe, a spojrzenia tak szydercze, iż łatwo było rycerzowi odgadnąć, że mu drogi nie wskażą, a jeżeli na pytanie odpowiedzą, to chyba grubiaństwem lub zniewagą. Niektórzy śmieli się, pokazując go sobie palcami; inni poczęli nań znów miotać śniegiem, tak samo jak dnia wczorajszego. Lecz on, spostrzegłszy drzwi większe od innych, nad którymi wykuty był w kamieniu Chrystus na krzyżu, udał się ku nim w mniemaniu, że jeśli komtur i starszyzna znajdują się w innej części zamku lub w innych izbach, to go ktoś przecie musi z błędnej drogi nawrócić.'
print(K.classifyFullProb(fragment))

{'adam-mickiewicz': 168, 'henryk-sienkiewicz': 1228, 'juliusz-slowacki': 42}


In [42]:
#Fragment książki Balladyna Słowackiego
fragment = '''Ty wiesz, jak nasza ziemia wszeteczeństwem
Króla skalana. Wiesz, jak Popiel krwawy
Pastwi się coraz nowym okrucieństwem…
Zaczerwienione krwią widziałem stawy:
Król żywi karpie ciałem niewolników.
Nieraz wybiera dziesiątego z szyków
I tnąc w kawały, ulubionym rybom
Na żer wyrzuca; resztę ciał wymiata'''

print(K.classifyFullProb(fragment))

{'adam-mickiewicz': 88, 'henryk-sienkiewicz': 336, 'juliusz-slowacki': 71}


In [43]:
#Fragment książki Balladyna Słowackiego
fragment = '''Nim Kirkor w przepaść okropną się rzuci
Szukając zemsty — chcę — chciałbym cię badać,
Na jakim pieńku zaszczepić rodowe
Drzewo Kirkorów, aby kiedyś nowe'''

print(K.classifyFullProb(fragment))

{'adam-mickiewicz': 0, 'henryk-sienkiewicz': 0, 'juliusz-slowacki': 113}


Teraz staramy się zklasyfikować losowe zdania z współczesnego świata

In [44]:
#Fragment wiersza Lokomytwa Tuwima
fragment = '''Stoi na stacji lokomotywa,
Ciężka, ogromna i pot z niej spływa:
Tłusta oliwa.
Stoi i sapie, dyszy i dmucha'''

print(K.classifyFullProb(fragment))

{'adam-mickiewicz': 0, 'henryk-sienkiewicz': 0, 'juliusz-slowacki': 0}


In [45]:
#Fragment książki Władcza Pierścieni Tolkiena
fragment = '''Gandalf rozmyślał o wiośnie sprzed prawie osiemdziesięciu lat, kiedy to Bilbo wybiegł z Bag End bez chusteczki do nosa. On sam miał teraz włosy chyba bielsze niż wówczas, brodę dłuższą i brwi bardziej krzaczaste, twarz pobrużdżoną od doznanych przeżyć, ale oczy żywe jak zawsze. Kurzył fajeczkę i wydmuchiwał kółka dymu z tym samym co dawniej zadowoleniem.'''

print(K.classifyFullProb(fragment))

{'adam-mickiewicz': 207, 'henryk-sienkiewicz': 1215, 'juliusz-slowacki': 103}


In [46]:
#Fragment filmu Forrest Gump
fragment = '''Moja mama zawsze mówiła: „Życie jest jak pudełko czekoladek. Nigdy nie wiesz, co ci się trafi.”'''
print(K.classifyFullProb(fragment))

{'adam-mickiewicz': 45, 'henryk-sienkiewicz': 0, 'juliusz-slowacki': 41}


### Wnioski

Patrząć na wyniki możemy dojść do naprawdę ciekawych obserwacji:

* Dla słow kluczowych np. imion głównych, lub drugoplanowych bohaterów książek klasyfikacja przebiega bardzo dobrze i prawidłowo wykrywa odpowiedniego autora. Dodatkowo klasyfikacja jest oczywista, praktycznie 
inni autorzy nie są nawet trochę podobni
* Wkładająć zdania z książek, też mamy w większości przypadków prawidłową klasyfikację, wyjątkiem jest jeden z fragmentów Balladyny, który został myli zklasyfikowany do Sienkiewicza, jest to prawdopodobnie spowodowane tym, że dany fragment nie była zbytnio charakterystyczny dla Balladyny a słowa, które wypowiadał były uniwersale np. dla tekstów sienkiewicza. Dodatkowo warto zauważyć, że teksty Sienkiewicza są bardzo
długie np. Krzyżacy, Potop — z tego też powodu znajdziemy tam ogromą gammę styłów, czasów i wypowiedzi bohaterów co mogło po prostu przyćmić tekst Balladyny
* Warto zauważyć, że dla innego fragmentu Balladyny w którym pojawia się "Kirkor" klasyfikacja przebiegła bardzo pomyślnie.
* Jeżeli chodzi o wykrywanie fragmentów z innych dzieł, również i współczesnych tutaj sytuacja jest bardzo różna. Mamy przypadki takie jak z "Lokomotywą" gdzie nie znaleźliśmy żadnego powiąznia, a mamy sytuację z fragmentem książki "Władca Pierścieni", gdzie powiązanie z Sienkiewiczem jest bardzo duże.

Podsumowując, możemy stwierdzić, że klasyfikator działa poprawnie, jednak warto używać fragmentu, który jest w miarę charakterystyczny dla danego autora, szczególnie jeżeli mamy 3 autorów żyjących mnie więcej w tym samym czasie.